# Caso Técnico Xepelin
# Data pre-processing 
#### Florencia Margara
#### Fecha: 2022-08-26

## Import Libs

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Configuración
# ==============================================================================
from IPython.display import display, HTML
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 500)

## Load data 

In [2]:
df = pd.read_csv('data/data_test(2).csv', parse_dates=['paidAt'])

In [3]:
## Change Data Types
df = df.astype({"PayerId": str, "ReceiverId": str, "invoiceId":str})

In [4]:
## Ya que trabajaremos con series de tiempo eliminamos valores con fechas nulas, que corresponden a los status = PAID y PROCESSING
df.dropna( axis = 0, how ='any',inplace = True,)

In [5]:
## Add new columns
df['flgFinanced'] = df['amountfinancedByXepelin'].apply(lambda x: 1 if x != 0 else 0)
df['period'] = df['paidAt'].dt.to_period('M').dt.to_timestamp()

In [6]:
df.head()

,PayerId,ReceiverId,invoiceId,paidAt,amount,amountfinancedByXepelin,status,flgFinanced,period
0,53,10,18660729,2022-04-23,1490.46,0.00,PAID,0,2022-04-01
1,93,11,18660730,2021-11-20,6418.28,624.48,PAID,1,2021-11-01
2,122,12,18660731,2021-10-19,27979.20,10520.15,PAID,1,2021-10-01
3,85,13,18660732,2021-11-13,183070.77,79421.63,PAID,1,2021-11-01
4,87,14,18660733,2022-03-23,20532.00,20532.00,PAID,1,2022-03-01


In [7]:
# Dataset Final
# ==============================================================================
## Como se solicita una predicción mensual, agrupamos los datos en esa frecuencia 
## Creamos nuevas columnas agregando a partir de los datos disponibles

dataset = df.groupby(['period']).agg(freq           =('invoiceId', 'count')
                                    ,freqFinanced   =('flgFinanced', 'sum')
                                    ,totalAmount    =('amount', 'sum')
                                    ,meanAmount     =('amount', 'mean')
                                    ,totalFinanced  =('amountfinancedByXepelin', 'sum')
                                    ,meanFinanced   =('amountfinancedByXepelin', 'mean')
                                    ,freqPayers     =('PayerId', lambda x: x.nunique())
                                    ,freqReceiver   =('ReceiverId', lambda x: x.nunique())
                                    )\
                                .reset_index()

In [8]:
dataset

,period,freq,freqFinanced,totalAmount,meanAmount,totalFinanced,meanFinanced,freqPayers,freqReceiver
0,2021-10-01,5,1,35902.64,7180.53,10520.15,2104.03,5,5
1,2021-11-01,13,8,268349.19,20642.25,119352.51,9180.96,13,12
2,2021-12-01,17,7,753946.49,44349.79,592831.07,34872.42,16,16
3,2022-01-01,107,43,7283045.52,68065.85,5219511.13,48780.48,83,56
4,2022-02-01,183,45,7100660.51,38801.42,4485712.42,24512.09,115,65
5,2022-03-01,225,49,5765920.25,25626.31,2344379.98,10419.47,137,68
6,2022-04-01,269,50,5042399.69,18744.98,3057870.38,11367.55,153,73
7,2022-05-01,73,18,1180788.62,16175.19,714735.77,9790.90,62,35


In [9]:
dataset.to_csv('data/data_final.csv')

In [10]:
# Dataset Final con rezago de un periodo AMOUNT
# ==============================================================================
## 

In [11]:
## Obtenemos el target
target = dataset.iloc[1:,[0,3]].reset_index().iloc[:,1:]
target

,period,totalAmount
0,2021-11-01,268349.19
1,2021-12-01,753946.49
2,2022-01-01,7283045.52
3,2022-02-01,7100660.51
4,2022-03-01,5765920.25
5,2022-04-01,5042399.69
6,2022-05-01,1180788.62


In [12]:
## Obtenemos los datos restantes con rezago de un periodo (del 10-2021 al 04-2022)
features = dataset.iloc[:7,[1,2,4,5,6,7,8]].reset_index().iloc[:,1:]
features

,freq,freqFinanced,meanAmount,totalFinanced,meanFinanced,freqPayers,freqReceiver
0,5,1,7180.53,10520.15,2104.03,5,5
1,13,8,20642.25,119352.51,9180.96,13,12
2,17,7,44349.79,592831.07,34872.42,16,16
3,107,43,68065.85,5219511.13,48780.48,83,56
4,183,45,38801.42,4485712.42,24512.09,115,65
5,225,49,25626.31,2344379.98,10419.47,137,68
6,269,50,18744.98,3057870.38,11367.55,153,73


In [13]:
dataset_lag = pd.concat([target,features], axis=1)
dataset_lag

,period,totalAmount,freq,freqFinanced,meanAmount,totalFinanced,meanFinanced,freqPayers,freqReceiver
0,2021-11-01,268349.19,5,1,7180.53,10520.15,2104.03,5,5
1,2021-12-01,753946.49,13,8,20642.25,119352.51,9180.96,13,12
2,2022-01-01,7283045.52,17,7,44349.79,592831.07,34872.42,16,16
3,2022-02-01,7100660.51,107,43,68065.85,5219511.13,48780.48,83,56
4,2022-03-01,5765920.25,183,45,38801.42,4485712.42,24512.09,115,65
5,2022-04-01,5042399.69,225,49,25626.31,2344379.98,10419.47,137,68
6,2022-05-01,1180788.62,269,50,18744.98,3057870.38,11367.55,153,73


In [14]:
dataset_lag.to_csv('data/data_final_lag_amount.csv')

In [15]:
# Dataset Final con rezago de un periodo AMOUNT FINANCED
# ==============================================================================
## 

In [16]:
## Obtenemos el target
target = dataset.iloc[1:,[0,5]].reset_index().iloc[:,1:]
target

,period,totalFinanced
0,2021-11-01,119352.51
1,2021-12-01,592831.07
2,2022-01-01,5219511.13
3,2022-02-01,4485712.42
4,2022-03-01,2344379.98
5,2022-04-01,3057870.38
6,2022-05-01,714735.77


In [17]:
## Obtenemos los datos restantes con rezago de un periodo (del 10-2021 al 04-2022)
features = dataset.iloc[:7,[1,2,3,4,6,7,8]].reset_index().iloc[:,1:]
features

,freq,freqFinanced,totalAmount,meanAmount,meanFinanced,freqPayers,freqReceiver
0,5,1,35902.64,7180.53,2104.03,5,5
1,13,8,268349.19,20642.25,9180.96,13,12
2,17,7,753946.49,44349.79,34872.42,16,16
3,107,43,7283045.52,68065.85,48780.48,83,56
4,183,45,7100660.51,38801.42,24512.09,115,65
5,225,49,5765920.25,25626.31,10419.47,137,68
6,269,50,5042399.69,18744.98,11367.55,153,73


In [18]:
dataset_lag = pd.concat([target,features], axis=1)
dataset_lag

,period,totalFinanced,freq,freqFinanced,totalAmount,meanAmount,meanFinanced,freqPayers,freqReceiver
0,2021-11-01,119352.51,5,1,35902.64,7180.53,2104.03,5,5
1,2021-12-01,592831.07,13,8,268349.19,20642.25,9180.96,13,12
2,2022-01-01,5219511.13,17,7,753946.49,44349.79,34872.42,16,16
3,2022-02-01,4485712.42,107,43,7283045.52,68065.85,48780.48,83,56
4,2022-03-01,2344379.98,183,45,7100660.51,38801.42,24512.09,115,65
5,2022-04-01,3057870.38,225,49,5765920.25,25626.31,10419.47,137,68
6,2022-05-01,714735.77,269,50,5042399.69,18744.98,11367.55,153,73


In [19]:
dataset_lag.to_csv('data/data_final_lag_amountFinanced.csv')